In [35]:
import cudaq
import numpy as np
import random
from pymatching import Matching

cudaq.set_target("nvidia")

N = 3
NUM_PHY_QUBITS = 2*N*N-1
N_ROUNDS = 0 # plus initial round

In [36]:
# Check matrices for X and Z syndromes
HX = np.array([
    [0,1,1,0,0,0,0,0,0], # syndrome X0 connected to data qubits 1 and 2
    [1,1,0,1,1,0,0,0,0], # syndrome X1 connected to data qubits 0, 1, 3, and 4
    [0,0,0,0,1,1,0,1,1], # ...
    [0,0,0,0,0,0,1,1,0]])
HZ = np.array([
    [1,0,0,1,0,0,0,0,0],
    [0,1,1,0,1,1,0,0,0],
    [0,0,0,1,1,0,1,1,0],
    [0,0,0,0,0,1,0,0,1]])
mx = Matching(HX, repetitions=N_ROUNDS, timelike_weights=999)
mz = Matching(HZ, repetitions=N_ROUNDS, timelike_weights=999)

In [37]:
def performRounds(numRounds):
    kernel = cudaq.make_kernel()
    qd = kernel.qalloc(N*N)
    qs = kernel.qalloc(N*N-1)

    for round in range(numRounds+1):

        # Step 1 - Reset all stabilizers
        if round > 0:
            for i in range(N*N-1):
                kernel.reset(qs[i])

        # Step 2 - Perform H on X-stabilizers
        kernel.h(qs[0])
        kernel.h(qs[1])
        kernel.h(qs[2])
        kernel.h(qs[3])

        # NE
        kernel.cx(qs[0], qd[2])
        kernel.cx(qs[1], qd[4])
        kernel.cx(qs[2], qd[8])
        kernel.cx(qd[3], qs[4])
        kernel.cx(qd[5], qs[5])
        kernel.cx(qd[7], qs[6])

        # NW
        kernel.cx(qs[0], qd[1])
        kernel.cx(qs[1], qd[3])
        kernel.cx(qs[2], qd[7])
        kernel.cx(qd[4], qs[5])
        kernel.cx(qd[6], qs[6])
        kernel.cx(qd[8], qs[7])

        # SE
        kernel.cx(qs[1], qd[1])
        kernel.cx(qs[2], qd[5])
        kernel.cx(qs[3], qd[7])
        kernel.cx(qd[0], qs[4])
        kernel.cx(qd[2], qs[5])
        kernel.cx(qd[4], qs[6])

        # SW
        kernel.cx(qs[1], qd[0])
        kernel.cx(qs[2], qd[4])
        kernel.cx(qs[3], qd[6])
        kernel.cx(qd[1], qs[5])
        kernel.cx(qd[3], qs[6])
        kernel.cx(qd[5], qs[7])

        kernel.h(qs[0])
        kernel.h(qs[1])
        kernel.h(qs[2])
        kernel.h(qs[3])

        # Measure (and save) the syndromes
        kernel.mz(qs, f"round{round}")

        # Apply a random error or two
        if round == 5 or round == 6:
            kernel.x(qd[random.randint(0, 8)])
    kernel.mz(qd, 'data')
    results = cudaq.sample(kernel, shots_count=1)
    return results

In [38]:
for iter in range(1):
    # Run quantum simulation
    results = performRounds(N_ROUNDS)
    
    # Get data qubit measurements
    results_data = results.get_register_counts("data")
    measZ = np.bitwise_xor.reduce(np.array([int(bit) for bit in results_data.most_probable()]))

    # Form syndromes by XOR'ing with previous round's syndrome measurements
    syndrome_x = list()
    syndrome_z = list()
    for round in range(1,N_ROUNDS+1):
        prevRound = results.get_register_counts(f"round{round-1}")
        prevRoundSyn = np.array([int(bit) for bit in prevRound.most_probable()])
        thisRound = results.get_register_counts(f"round{round}")
        thisRoundSyn = np.array([int(bit) for bit in thisRound.most_probable()])
        syndromes = np.bitwise_xor(prevRoundSyn, thisRoundSyn)
        syndrome_x.append(syndromes[0:4])
        syndrome_z.append(syndromes[4:])
    syndrome_x = np.array(syndrome_x)
    syndrome_z = np.array(syndrome_z)

    # Decode syndromes and apply corrections to data
    decode_x = mx.decode(syndrome_x.transpose())
    decode_z = mz.decode(syndrome_z.transpose())
    resX = np.bitwise_xor.reduce(decode_x)
    resZ = np.bitwise_xor.reduce(decode_z)
    correctedZ = measZ ^ resZ
    print("Iter", iter, decode_x, decode_z, resX, resZ, f"meas is {measZ} and corrected is {correctedZ}")

Iter 0 [0 0 0 0 0 0 0 0 0] [0 0 0 0 0 0 0 0 0] 0 0 meas is 0 and corrected is 0
